<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/4_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

# Dataset import

In [3]:
from sklearn import preprocessing

data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/data.csv')
data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

min_max_scaler = preprocessing.MinMaxScaler()
time = data['TransactionDT'].values.reshape(-1,1)
data['TransactionDT'] = min_max_scaler.fit_transform(time)
data.drop(columns=['TransactionID'], inplace=True)

In [4]:
X_train = mf.get_set('train_id', data, 'catboost_new')
X_val = mf.get_set('val_id', data, 'catboost_new')
X_test = mf.get_set('test_id', data, 'catboost_new')

# Model import

### Catboost 

Riferimento: https://stackoverflow.com/questions/30483246/how-to-check-if-a-python-module-has-been-imported

In [5]:
load_catboost = True
if load_catboost:
  if 'catboost' not in sys.modules:
    !pip install catboost 
  from catboost import CatBoostClassifier
  cat = CatBoostClassifier()
  cat.load_model('/content/drive/MyDrive/Tesi_magistrale/Dataset/IEEE/Output/modelli/catboost_new')  
  categorical_col=mf.load_list('cat_sign_col')

# LIME

**LIME Local interpretable model-agnostic explanations**

1. LIME generates a new dataset consisting of: 
  * permuted samples, drawing from a normal distribution with mean and standard deviation taken from the feature (in case of tabular data);
  * the corresponding predictions of the black box model;
2. On this new dataset LIME then trains an interpretable model;
3. The model is weighted by the proximity of the sampled instances to the instance of interest.

The **recipe for training local surrogate models**:
1. Select your instance of interest for which you want to have an explanation of its black box prediction.
2. Perturb your dataset and get the black box predictions for these new points.
3. Weight the new samples according to their proximity to the instance of interest.
4. Train a weighted, interpretable model on the dataset with the variations.
5. Explain the prediction by interpreting the local model.

Tutorial LIME: https://marcotcr.github.io/lime/tutorials/Tutorial%20-%20continuous%20and%20categorical%20features.html

Documentazione LIME: https://lime-ml.readthedocs.io/en/latest/lime.html

Repository github di lime: https://github.com/marcotcr/lime

In [ ]:
!pip install lime

Riferimento: https://stackoverflow.com/questions/13021654/get-column-index-from-column-name-in-python-pandas

In [7]:
categorical_features = [X_train.columns.get_loc(col) for col in categorical_col if col in X_train]

In [17]:
categorical_col

['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'id_12',
 'id_15',
 'id_16',
 'id_23',
 'id_27',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo']

Riferimento: https://stackoverflow.com/questions/52404971/get-a-list-of-categories-of-categorical-variable-python-pandas

In [ ]:
categorical_names=[]
for col in categorical_col:
  values = X_train[col].unique()
  for val in values:
    if val not in categorical_names:
      categorical_names.append(val)

# categorical_names

In [19]:
def lime(X_train, X_test, categorical_features, categorical_names, clf, n_features, j):
  import lime
  import lime.lime_tabular

  # setup dell'esxplainer
  explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns.values.tolist(),
                                                      class_names=[0,1],
                                                      categorical_features=categorical_features,
                                                      categorical_names=categorical_names,
                                                      verbose=True, mode='classification',
                                                      discretize_continuous = True, discretizer='decile')
  
  # explanation
  exp = explainer.explain_instance(X_test.values[j], clf.predict_proba, num_features=n_features)
  exp.show_in_notebook(show_table=True)
  return  

In [20]:
lime(X_train, X_test, categorical_features, categorical_names, cat, 10, 0)

ValueError: ignored